In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from bs4 import BeautifulSoup 
import requests
from datetime import datetime as dt
import plotly.subplots as sp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.tseries.offsets import MonthEnd

In [2]:
from zipfile import ZipFile
from io import BytesIO
from urllib.request import urlopen

from RGGI_plant_analysis import RGGI_capacity

In [3]:
### Capacity
def save_historical_capacity_per_plant():
    time_series_historical_capacity = pd.DataFrame()
    for x in range(1,12*6):
        next_update_time,recent_report,report_month,report_year  =RGGI_capacity().scrape_recent_EIA_860m(lagged_report=x)
        capacity = RGGI_capacity().analyse_all_capacity_with_tech(recent_report,report_month,report_year)
        time_series_historical_capacity = pd.concat([time_series_historical_capacity,capacity],axis=0)

    time_series_historical_capacity.to_pickle('full_capacity_series_with_tech.pkl')

    return time_series_historical_capacity

In [4]:
time_series_historical_capacity_with_tech = pd.read_pickle('full_capacity_series_with_tech.pkl')  #= save_historical_capacity_per_plant()

In [5]:
time_series_historical_capacity_with_tech = time_series_historical_capacity_with_tech.reset_index()

In [6]:
RGGI_states = ['CT', 'DE', 'ME', 'MD', 'MA', 'NH', 'NJ', 'NY', 'RI', 'VT'] 




In [9]:
tech_convert_dict = {'Petroleum Liquids':'Oil',
                    'Onshore Wind Turbine':'Wind',
       'Conventional Hydroelectric':'Hydro', 
       'Conventional Steam Coal':'Coal',
       'Natural Gas Fired Combined Cycle':'Gas', 
       'Natural Gas Steam Turbine':'Gas',
       'Natural Gas Fired Combustion Turbine':'Gas', 
       'Nuclear':'Nuclear',
       'Hydroelectric Pumped Storage':'Storage',
       'Natural Gas Internal Combustion Engine':'Gas', 
       'Batteries':'Storage',
       'Solar Photovoltaic':'Solar', 
       'Geothermal':'Other Renewables', 
       'Wood/Wood Waste Biomass':'Other Renewables',
       'Coal Integrated Gasification Combined Cycle':'Gas',
        'Other Gases':'Gas',
       'Petroleum Coke':'Coal', 
       'Municipal Solid Waste':'Other', 
       'Landfill Gas':'Other Renewables',
       'Natural Gas with Compressed Air Storage':'Gas', 
       'All Other':'Other',
       'Other Waste Biomass':'Other Renewables', 
       'Solar Thermal without Energy Storage':'Solar',
       'Other Natural Gas':'Gas', 
       'Solar Thermal with Energy Storage':'Solar',
       'Flywheels':'Other', 
       'Offshore Wind Turbine':'Wind', 
       'Hydrokinetic':'Hydro',
       'Other Energy Storage':'Storage'}

In [10]:
Fossil_tech = ['Coal','Gas','Oil']

In [12]:
time_series_historical_capacity_with_tech['PJM_tech'] = time_series_historical_capacity_with_tech['Technology'].map(tech_convert_dict)

In [13]:
time_series_historical_capacity_with_tech['Fossil'] = np.where(time_series_historical_capacity_with_tech['PJM_tech'].isin(Fossil_tech),1,0)

In [15]:
dual_fuels = time_series_historical_capacity_with_tech.groupby(['Plant ID','report_month','report_year'])['Fossil'].sum()

In [17]:
time_series_historical_capacity_with_tech = time_series_historical_capacity_with_tech.join(dual_fuels.to_frame('dual_fossil'),on=['Plant ID','report_month','report_year'])

In [18]:
time_series_historical_capacity_with_tech['PJM_tech'] = np.where(time_series_historical_capacity_with_tech['dual_fossil']>1,'Multiple Fuels',time_series_historical_capacity_with_tech['PJM_tech'])

In [19]:
time_series_historical_capacity_with_tech['RGGI_state'] = np.where(time_series_historical_capacity_with_tech['Plant State'].isin(RGGI_states),1,0)

In [20]:
time_series_historical_capacity_with_tech['RGGI_eligible'] = np.where((time_series_historical_capacity_with_tech['RGGI_state']==1) & (time_series_historical_capacity_with_tech['Nameplate Capacity (MW)']>=25),1,0)

In [21]:
#tCO2/MWh
simple_emissions_factor={'Coal': 1.046,
                           "Gas": 0.467,
                           "Oil": 0.867,
                           'Multiple Fuels':0.467*0.9+0.867*.1}

metric_tons_to_short_tons = 1.1023


In [22]:
time_series_historical_capacity_with_tech['rough_EF'] = time_series_historical_capacity_with_tech['PJM_tech'].map(simple_emissions_factor)

In [23]:
time_series_historical_capacity_with_tech['rough_EF']  =time_series_historical_capacity_with_tech['rough_EF'].fillna(0)

In [24]:
time_series_historical_capacity_with_tech['rough_emissions'] =time_series_historical_capacity_with_tech['rough_EF']*time_series_historical_capacity_with_tech['Nameplate Capacity (MW)']

In [25]:
pjm_rggi_capacity_compare = time_series_historical_capacity_with_tech.query('`Balancing Authority Code`=="PJM"').groupby(['report_year','report_month','PJM_tech','RGGI_eligible'])['Nameplate Capacity (MW)'].sum().unstack('RGGI_eligible')
pjm_rggi_capacity_compare = pjm_rggi_capacity_compare.reset_index()
pjm_rggi_capacity_compare['Date'] = pd.to_datetime('01/'+pjm_rggi_capacity_compare['report_month'].astype(str)+'/'+pjm_rggi_capacity_compare['report_year'].astype(str),format='%d/%m/%Y')+MonthEnd(0)

In [41]:
pjm_rggi_emissions_compare = time_series_historical_capacity_with_tech.query('`Balancing Authority Code`=="PJM"').groupby(['report_year','report_month','PJM_tech','RGGI_eligible'])['rough_emissions'].sum().unstack('RGGI_eligible')
pjm_rggi_emissions_compare = pjm_rggi_emissions_compare.reset_index()
pjm_rggi_emissions_compare['Date'] = pd.to_datetime('01/'+pjm_rggi_emissions_compare['report_month'].astype(str)+'/'+pjm_rggi_emissions_compare['report_year'].astype(str),format='%d/%m/%Y')+MonthEnd(0)

In [42]:
pjm_rggi_capacity_compare['RGGI_share'] = pjm_rggi_capacity_compare[1]/pjm_rggi_capacity_compare[[1,0]].sum(axis=1)

In [43]:
pjm_rggi_capacity_compare.groupby(['report_year','report_month','Date','PJM_tech'])['RGGI_share'].mean().to_frame().fillna(0).to_csv('RGGI_share_PJM.csv')

In [47]:
pjm_rggi_capacity_compare.groupby(['report_year','report_month','Date','PJM_tech'])['RGGI_share'].mean().to_frame().fillna(0)

RGGI_share
report_year report_month Date       PJM_tech                    
2018        11           2018-11-30 Coal                0.050723
                                    Gas                 0.114144
                                    Hydro               0.160426
                                    Multiple Fuels      0.283955
                                    Nuclear             0.154229
...                                                          ...
2024        9            2024-09-30 Other               0.375133
                                    Other Renewables    0.000000
                                    Solar               0.020297
                                    Storage             0.083040
                                    Wind                0.015965

[781 rows x 1 columns]

In [46]:
pjm_rggi_capacity_compare.query('report_year==2024 and report_month==9')

RGGI_eligible,report_year,report_month,PJM_tech,0,1,Date,RGGI_share
770,2024,9,Coal,29723.8,1370.2,2024-09-30,0.044066
771,2024,9,Gas,38550.7,3059.7,2024-09-30,0.073532
772,2024,9,Hydro,2808.6,530.8,2024-09-30,0.158951
773,2024,9,Multiple Fuels,56260.0,18081.7,2024-09-30,0.243224
774,2024,9,Nuclear,28985.5,5481.1,2024-09-30,0.159026
775,2024,9,Oil,3047.8,649.2,2024-09-30,0.175602
776,2024,9,Other,469.9,282.1,2024-09-30,0.375133
777,2024,9,Other Renewables,1306.4,NaN,2024-09-30,NaN
778,2024,9,Solar,12506.5,259.1,2024-09-30,0.020297
779,2024,9,Storage,5002.2,453.0,2024-09-30,0.083040


In [39]:
pjm_rggi_capacity_compare.query(f'report_year>={dt.today().year-1}').groupby(['report_month','PJM_tech'])['RGGI_share'].mean().fillna(0).to_frame('recent_RGGI_share').loc[11]

,recent_RGGI_share
PJM_tech,
Coal,0.047396
Gas,0.076174
Hydro,0.158951
Multiple Fuels,0.249281
Nuclear,0.159026
Oil,0.203091
Other,0.347414
Other Renewables,0.000000
Solar,0.028829


In [29]:
px.bar(pjm_rggi_capacity_compare.groupby(['PJM_tech','report_year','report_month','Date'])['RGGI_share'].mean().to_frame().reset_index(),x='Date',y='RGGI_share',color='PJM_tech',barmode='group')

In [30]:
fig = make_subplots()
fig.add_trace(go.Bar(x=pjm_rggi_capacity_compare.query('PJM_tech=="Coal"').Date,y=pjm_rggi_capacity_compare.query('PJM_tech=="Coal"')[[0,1]].sum(axis=1),name='Non RGGI PJM'))
fig.add_trace(go.Bar(x=pjm_rggi_capacity_compare.query('PJM_tech=="Coal"').Date,y=pjm_rggi_capacity_compare.query('PJM_tech=="Coal"')[1],name='RGGI PJM'))

In [31]:
pjm_rggi_capacity_compare

RGGI_eligible,report_year,report_month,PJM_tech,0,1,Date,RGGI_share
0,2018,11,Coal,41010.2,2191.3,2018-11-30,0.050723
1,2018,11,Gas,29257.8,3769.9,2018-11-30,0.114144
2,2018,11,Hydro,2777.9,530.8,2018-11-30,0.160426
3,2018,11,Multiple Fuels,56347.3,22345.1,2018-11-30,0.283955
4,2018,11,Nuclear,29938.5,5459.4,2018-11-30,0.154229
...,...,...,...,...,...,...,...
776,2024,9,Other,469.9,282.1,2024-09-30,0.375133
777,2024,9,Other Renewables,1306.4,NaN,2024-09-30,NaN
778,2024,9,Solar,12506.5,259.1,2024-09-30,0.020297
779,2024,9,Storage,5002.2,453.0,2024-09-30,0.083040


In [32]:
fig = make_subplots()
fig.add_trace(go.Bar(x=pjm_rggi_capacity_compare.query('PJM_tech=="Gas"').Date,y=pjm_rggi_capacity_compare.query('PJM_tech=="Gas"')[[0,1]].sum(axis=1),name='Non RGGI PJM'))
fig.add_trace(go.Bar(x=pjm_rggi_capacity_compare.query('PJM_tech=="Gas"').Date,y=pjm_rggi_capacity_compare.query('PJM_tech=="Gas"')[1],name='RGGI PJM'))